# I attempt to compile a lot of sequential monte carlo (SMC) notes with some toy examples

In [76]:
import autograd.numpy as np
from autograd import grad
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from simtk import unit
kB = unit.BOLTZMANN_CONSTANT_kB #* unit.AVOGADRO_CONSTANT_NA
TEMPERATURE=300*unit.kelvin
kT = kB * TEMPERATURE
MASS= 6 * unit.daltons
DT=2.0*unit.femtoseconds

#propagation parameters
beta = 1/kT
tau = (DT**2)*kT/(2*MASS)

(DT**2*kT).value_in_unit(unit.daltons * unit.nanometers**2/unit.femtoseconds**2)

TypeError: Unit "femtosecond**2*joule" is not compatible with Unit "nanometer**2*dalton/(femtosecond**2)".

## First, let's get the most basic of the basic theory out of the way for importance sampling (IS)...
we need to compute the expectation of some generic function $h_t$ with respect to the target $\gamma_t$,
$$
\gamma_t(h_t) \equiv \mathbf{E}_{\gamma_t} \left[ h_t(x_{1:t}) \right]
$$
which can be rewritten as
$$
\mathbf{E}_{\gamma_t} \left[ h_t(x_{1:t}) \right]
=
\frac{1}{Z_t}\mathbf{E}_{q_t}
\left[
\frac{\tilde{\gamma_t}(x_{1:t})}{q_t(x_{1:t})} h_t(x_{1:t})
\right] 
= 
\frac{\mathbf{E}_{q_t}
\left[
\frac{\tilde{\gamma_t}(x_{1:t})}{q_t(x_{1:t})} h_t(x_{1:t})
\right]}
{\mathbf{E}_{q_t}
\left[
\frac{\tilde{\gamma_t}(x_{1:t})}{q_t(x_{1:t})}
\right]}
$$.
$q_t$ is a user-chosen proposal distribution that is hopefully simple to sample from and evaluate.
The RHS can be estimated with MC as 
$$
\mathbf{E}_{\gamma_t} \left[ h_t(x_{1:t}) \right]
\approx
\frac
{\frac{1}{N} \sum_{i=1}^N \widetilde{w}_t(x_{1:t}^i) h_t(x_{1:t}^i)}
{\frac{1}{N} \sum_{j=1}^N \widetilde{w}_t(x_{1:t}^j)}
$$
where $\widetilde{w}_t(x_{1:t}) \equiv \frac{\widetilde{\gamma}_t(x_{1:t})} {q_t(x_{1:t})}$ and $x_{1:t}^i$ are simulated i.i.d. from $q_t$. 
We can express
$$
\mathbf{E}_{\gamma_t} \left[ h_t(x_{1:t}) \right] \approx \sum_{i=1}^N w_t^i h_t(x_{1:t}^i), \quad x_{1:t}^i \sim q_t
$$
with 
$$
w_t^i \equiv \frac{\widetilde{w}_t^i} {\sum_j \widetilde{w}_t^j}
$$.
We can also write the empirical distribution of $\gamma_t(x_{1:t})$ as
$$
\gamma_t(x_{1:t}) \approx \sum_{i=1}^N w_t^i \delta_{x_{1:t}^i}(x_{1:t}) \equiv \widehat{\gamma}_t(x_{1:t})
$$.
Finally,
$$
Z_t \approx \frac{1}{N} \sum_{i=1}^N \widetilde{w}_t^i \equiv \widehat{Z}_t
$$,
and it can easily be shown that $\mathbf{E} \left[ \widehat{Z}_t\right] \equiv Z_t$


## There is one modification to be made for Sequential Importance Sampling (SIS)
SIS makes IS 'autoregressive'.  That is, if we choose a proposal defined as 
$$
q_t(x_{1:t}) = q_{t-1}(x_{1:t-1})q_t(x_t | x_{1:t-1})
$$
then we can decompose $\widetilde{w}_t$ as 
$$
\widetilde{w}_t(x_{1:t}) = \frac{\widetilde{\gamma}_t(x_{1:t})} {q_t(x_{1:t})}
=
\frac{\widetilde{\gamma}_{t-1}(x_{1:t-1})} {q_{t-1}(x_{1:t-1})} \frac{\widetilde{\gamma}_t(x_{1:t})} 
{\widetilde{\gamma}_{t-1}(x_{1:t-1}) q_t(x_t | x_{1:t-1})}
\\
=\widetilde{w}_{t-1}(x_{1:t-1}) \frac{\widetilde{\gamma}_t(x_{1:t})} 
{\widetilde{\gamma}_{t-1}(x_{1:t-1}) q_t(x_t | x_{1:t-1})}
$$

Before we proceed with a 'special' case of SIS, we will demonstrate the derivation of the quantity that we are interested in, $Z_t/Z_{t-1}$.
$$
\frac{Z_t} {Z_{t-1}} = 
\frac{1}{Z_{t-1}}
\int \frac{\widetilde{\gamma}_{t-1}(x_{1:t-1}) \widetilde{\gamma}_t(x_{1:t})}
{q_{t-1}(x_{1:t-1}) \widetilde{\gamma}_{t-1}(x_{1:t}) q_t(x_t | x_{1:t-1})}
q_t(x_{1:t}) dx_{1:t}
\\
= \mathbf{E}_{\gamma_{t-1}(x_{1:t-1}) q_t (x_t | x_{1:t-1})}
\left[
\frac{ \widetilde{\gamma}_t(x_{1:t})}
{q_{t-1}(x_{1:t-1}) \widetilde{\gamma}_{t-1}(x_{1:t})}
\right]
\approx
\sum_{i=1}^N w_{t-1}^i \frac{\widetilde{w}_t^i} {\widetilde{w}_{t-1}^i}
=
\frac{\sum_{i=1}^N \widetilde{w}_t^i}
{\sum_{j=1}^N \widetilde{w}_t^j}
$$

## Now, we will consider a special case of SIS
We are going to define the distribution
$$
\pi_t(x) = \rho_t(x)/Q_t
$$
so that we can define an expectation
$$
\mathbf{E}_{\pi_t}(\phi) = Q_t^{-1} \int \phi(x) w_t(x) q_t(x) dx
$$
and 
$$
Q_t = \int w_t(x)q_t(x)dx
$$
where $w_t(x) = \rho_t(x) / q_t(x)$.
We need to create a marginal distribution that is defined by 
$$
\nu q_{i:j}(x_j) \equiv
\int \nu(x_{i-1}) \prod_{k=i}^j q_t(x_k | x_{k-1}) dx_{i-1:j-1}
$$.
Now, we refer back to the joint artificial target
$$
\gamma(x_{1:t}) = \frac{\widetilde{\gamma}_t(1:t)} {Z_t}
$$
where 
$$
\widetilde{\gamma}_t(x_{1:t}) = \rho_t(x_t) \prod _{k=1}^{t-1} L_k(x_{k+1}, x_k)
$$
and 




In [65]:
from openmmtools import testsystems
from openmmtools.states import ThermodynamicState
from simtk import unit
temperature = 298.0*unit.kelvin
waterbox = testsystems.WaterBox(box_edge=10*unit.angstroms,
                                 cutoff=4*unit.angstroms).system
state = ThermodynamicState(system=waterbox, temperature=temperature)

In [66]:
def V(position, _lambda):
    return (position - _lambda)**2/2

In [67]:
#let's make a function that conducts an euler maruyama move in 1 dimension
def euler_maruyama_compute_dx_grad(old_positions, potential, _lambda):
    """
    compute the deterministic update portion of an euler_maruyama integrator
    
    arguments
        old_positions : float (distance) (presumed in nm)
        potential : function (return float, presumed in units kT)
        potential_kwargs : other arguments to the potential function
    
    return 
        dx_grad : quantity (distance)
            the gradient update (deterministic)
    """
    #force is implicitly in kT/nanometers
    force = -1 * grad(potential, 0)(old_positions, _lambda) #this takes the gradient of the potential with respect to the first argument (positions) conditioned on all of the other arguments (e.g. alchemical parameters)
    force_unitd = force * kT / unit.nanometers
    
    #this is the deterministic part of dx
    dx_grad = tau * force_unitd * beta
    
    return dx_grad
    
def euler_maruyama_propagation(old_positions, potential, _lambda):
    """
    The update scheme works as follows:
    x_(k+1) = x_k + (tau) * force(x_k | parameters) * beta + sqrt(2*(tau)) * R(t)
    
    where:
        force(x_k | parameters) = -grad(potential),
        tau = D * dt; [tau] = positions_unit ** 2
        D = 1 / (2 * mass/kT); [D] = velocity_unit ** 2
    
    dx_grad = tau * force * beta #this is the incremental x update from the gradient of the distribution

    #now we add the stochastic part
    dx_stochastic = np.random.randn(self.mass_vector.shape[1], 3) * np.sqrt(2 * tau.value_in_unit(self.x_unit**2)) * unit.x_unit

    #combine them...
    dx = dx_grad + dx_stochastic

    proposal_work_numerator = -np.sum((new_positions - old_positions - tau * old_force * self.openmm_pdf_state.beta)**2) / (4.0 * tau)
    proposal_work_denominator = -np.sum((old_positions - new_positions - tau * new_force * self.openmm_pdf_state.beta)**2) / (4.0 * tau)
    
    arguments
        old_positions : float (distance) (presumed in nm)
        potential : function (return float, presumed in units kT)
        new_positions : float (distance) (presumed in nm), default None
    
    returns
        new_positions : unit.quantity (distance)
        log_probability : float
    """
    dx_grad = euler_maruyama_compute_dx_grad(old_positions, potential, _lambda)
    
    #this is the stochastic part of dx
    tau_unitless = 
    dx_stoch = np.random.randn() * np.sqrt(2.0 * tau.value_in_unit(unit.nanometers**2)) * unit.nanometers
    
    dx = dx_grad + dx_stoch
    
    new_positions = (old_positions * unit.nanometers + dx).value_in_unit(unit.nanometers)
    
    #compute the proposal
    log_proposal = euler_maruyama_proposal(old_positions, new_positions, dx_grad)
    
    return new_positions, log_proposal
    
    
    
def euler_maruyama_proposal(old_positions, new_positions, dx_grad):
    """
    compute the proposal of the euler_maruyama integrator...
    q(x' | x) \propto exp(-|| x' - x - dx_grad ||^2_2)
    we also remark that dx_grad = tau * force * beta #this is the incremental x update from the gradient of the distribution
    
    arguments
        old_positions : simtk.unit.quantity (distance)
            old positions
        new_positions : simtk.unit.quantity (distance)
            new positions
        dx_grad : simtk.unit.quantity (distance)
            deterministic update of old positions
    returns
        log_proposal : float
            return the proportional log likelihood of the proposal
    """
    proposal_exponand = (new_positions - old_positions - dx_grad).value_in_unit(unit.nanometers)
    log_proposal = -proposal_exponand**2
    return log_proposal
    
    

In [68]:
new_pos, log_p = euler_maruyama_propagation(0., V, 0.)

TypeError: Unit "femtosecond**2*joule/dalton" is not compatible with Unit "nanometer**2".

In [5]:
# def langevin(x0,
#              v0,
#              force,
#              n_steps=100,
#              stepsize=1 * unit.femtosecond,
#              collision_rate=10/unit.picoseconds,
#              temperature=300 * unit.kelvin,
#              progress_bar=False
#             ):
#     """Unadjusted Langevin dynamics.
    
#     Parameters
#     ----------
#     x0 : array of floats, unit'd (distance uniat)
#         initial configuration
#     v0 : array of floats, unit'd (distance unit / time unit)
#         initial velocities
#     force : callable, accepts a unit'd array and returns a unit'd array
#         assumes input is in units of distance
#         output is in units of energy / distance
#     n_steps : integer
#         number of Langevin steps
#     stepsize : float > 0, in units of time
#         finite timestep parameter
#     collision_rate : float > 0, in units of 1/time
#         controls the rate of interaction with the heat bath
#     temperature : float > 0, in units of temperature
#     progress_bar : bool
#         use tqdm to show progress bar
        
#     Returns
#     -------
#     traj : [n_steps + 1 x dim] array of floats, unit'd
#         trajectory of samples generated by Langevin dynamics
        
#     Note
#     ----
#         Assumes all particles have equal mass for convenience
#         (Hardcoded to 12 daltons, ignores the global variable `mass`!)
#     """
    
#     # x,v are input using units
    
#     assert(type(x0) == unit.Quantity)
#     assert(type(v0) == unit.Quantity)
#     assert(type(stepsize) == unit.Quantity)
#     assert(type(collision_rate) == unit.Quantity)
#     assert(type(temperature) == unit.Quantity)
    
#     # convert initial state numpy arrays with correct attached units
#     x = np.array(x0.value_in_unit(distance_unit)) * distance_unit
#     v = np.array(v0.value_in_unit(speed_unit)) * speed_unit
    
#     # traj is accumulated as a list of arrays with attached units
#     traj = [x]
    
#     # defining particles to have uniform masses
#     mass = 12.0 * unit.dalton # TODO: allow non-uniform masses?

#     # dimensionless scalars
#     a = np.exp(- collision_rate * stepsize)
#     b = np.sqrt(1 - np.exp(-2 * collision_rate * stepsize))
    
#     # temperature-dependent constants
#     kT = temperature * kB
#     sigma_v = np.sqrt(kT / mass) 
    
#     # compute force on initial configuration
#     F = force(x)
    
#     trange = range(n_steps)
#     if progress_bar:
#         trange = tqdm(trange)
#     for _ in trange:
#         # v
#         v += (stepsize * 0.5) * F / mass
#         # r
#         x += (stepsize * 0.5) * v
#         # o
#         v = (a * v) + (b * sigma_v * np.random.randn(*x.shape))
#         # r
#         x += (stepsize * 0.5) * v

#         F = force(x)
#         # v
#         v += (stepsize * 0.5) * F / mass
        
        
#         norm_F = np.linalg.norm(F)
        
#         # report gradient norm
#         if progress_bar:
#             trange.set_postfix({'|force|': norm_F})
        
#         # check positions and forces are finite
#         if (not np.isfinite(x).all()) or (not np.isfinite(norm_F)):
#             print("Numerical instability encountered!")
#             return traj

#         traj.append(x)

#     return traj